In [2]:
import numpy as np
import json
from utilities import *

In [6]:
inputJsonFileName = "test"
classJsonFileName = "classes"

samplingRate = 20e6
NRDLTXRate = 30.72e6

accStartIndexErr = 2
wlanHEExtPreambleLen = 80
wlanHEMedPreambleLen = 112
wlanHENorPreambleLen = 208

In [7]:
inputJsonFile = open("./inputJson/" + inputJsonFileName + ".json")
classJsonFile = open("./inputJson/class/" + classJsonFileName + ".json")
classJson = json.load(classJsonFile)

dataPath = json.load(inputJsonFile)["data_path"]

SNRVec = np.asarray(classJson["SNRVec"])
tauVec = np.asarray(classJson["tauVec"])
protocolList = np.asarray(classJson["protocolList"])
CPOptList = np.asarray(classJson["CPOptList"], dtype=object)
for i, CPLenElem in enumerate(CPOptList):
    CPOptList[i] = np.asarray(CPLenElem, dtype=object)
CPLenList = np.asarray(classJson["CPLenList"], dtype=object)

In [36]:
fileCount = np.zeros((len(protocolList),), dtype=object)
corrNSubCCount = np.zeros((len(protocolList),), dtype=object)
corrCPLenCount = np.zeros((len(protocolList),), dtype=object)
corrStartIndexCount = np.zeros((len(protocolList),), dtype=object)
corrSymIndexCount = np.zeros((len(protocolList),), dtype=object)
resultList = []
for i, protocol in enumerate(protocolList):
    fileCount[i] = np.zeros((len(SNRVec), CPOptList[i].shape[0]))
    corrNSubCCount[i] = np.zeros((len(SNRVec), ))
    corrCPLenCount[i] = np.zeros((len(SNRVec), CPOptList[i].shape[0]))
    corrStartIndexCount[i] = np.zeros((len(SNRVec), CPOptList[i].shape[0]))
    corrSymIndexCount[i] = np.zeros((len(SNRVec), CPOptList[i].shape[0]))
    if "NRDL" in protocol:
        tauVec[i] = (tauVec[i] * samplingRate / NRDLTXRate).astype(int)

loadArr = np.load('./result/230707_002050/result.npy', allow_pickle=True)
print(len(loadArr))

for iDir in range(len(loadArr)):
    for iFile in range(loadArr[iDir].shape[0]):
        fileSNR = int(loadArr[iDir][iFile][0].split('_')[4])
        fileProtocolTrue = loadArr[iDir][iFile][0].split('_')[0]
        fileCPOptTrue = loadArr[iDir][iFile][0].split('_')[1]

        fileSNRIndex = np.where(SNRVec == fileSNR)[0]
        fileProtocolIndex = np.where(protocolList == fileProtocolTrue)[0][0]
        fileCPOptIndex = np.where(CPOptList[fileProtocolIndex] == fileCPOptTrue)[0][0]

        fileCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
        if fileProtocolTrue == 'wlanHT':
            if loadArr[iDir][iFile][1] == 64:
                corrNSubCCount[fileProtocolIndex][fileSNRIndex] += 1
                if fileCPOptTrue == 'Extended':
                    if loadArr[iDir][iFile][2] == 16:
                        corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
                    startIndexTrue = (-loadArr[iDir][iFile][4]) % (64+16)
                elif fileCPOptTrue == 'Normal':
                    if loadArr[iDir][iFile][2] == 8:
                        corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
                    startIndexTrue = (-loadArr[iDir][iFile][4]) % (64+8)
                    # print(startIndexTrue, loadArr[iDir][iFile][3])
        
        elif fileProtocolTrue == 'wlanHE':
            if loadArr[iDir][iFile][1] == 256:
                corrNSubCCount[fileProtocolIndex][fileSNRIndex] += 1
                if fileCPOptTrue == 'Extended':
                    if loadArr[iDir][iFile][2] == 64:
                        corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
                    startIndexTrue = (-loadArr[iDir][iFile][4] - wlanHEExtPreambleLen) % (256+64)

                elif fileCPOptTrue == 'Medium':
                    if loadArr[iDir][iFile][2] == 32:
                        corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
                    startIndexTrue = (-loadArr[iDir][iFile][4] - wlanHEMedPreambleLen) % (256+32)

                elif fileCPOptTrue == 'Normal':
                    if loadArr[iDir][iFile][2] == 16:
                        corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
                    startIndexTrue = (-loadArr[iDir][iFile][4] - wlanHENorPreambleLen) % (256+16)

        elif fileProtocolTrue == 'NRDLa':
            if loadArr[iDir][iFile][1] == 333:
                corrNSubCCount[fileProtocolIndex][fileSNRIndex] += 1
                if fileCPOptTrue == 'Extended':
                    if loadArr[iDir][iFile][2] == 83:
                        corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
                    [startIndexTrue, symIndexTrue] = findTrueFirstIndexNR(loadArr[iDir][iFile][4], 512, 128)
                    # print('Extended:', loadArr[iDir][iFile][2])
                elif fileCPOptTrue == 'Normal':
                    if loadArr[iDir][iFile][2] == 23:
                        corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
                    [startIndexTrue, symIndexTrue] = findTrueFirstIndexNR(loadArr[iDir][iFile][4], 512, 36)
                    # print('Normal:', loadArr[iDir][iFile][2])
                # corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
            # print(fileCPOptTrue, startIndexTrue, loadArr[iDir][iFile][3], loadArr[iDir][iFile][4], symIndexTrue, loadArr[iDir][iFile][5])

        elif fileProtocolTrue == 'NRDLb':
            if loadArr[iDir][iFile][1] == 666:
                corrNSubCCount[fileProtocolIndex][fileSNRIndex] += 1
                corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
            [startIndexTrue, symIndexTrue] = findTrueFirstIndexNR(loadArr[iDir][iFile][4], 1024, 72)
        
        elif fileProtocolTrue == 'NRDLc':
            if loadArr[iDir][iFile][1] == 1333:
                corrNSubCCount[fileProtocolIndex][fileSNRIndex] += 1
                corrCPLenCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
            [startIndexTrue, symIndexTrue] = findTrueFirstIndexNR(loadArr[iDir][iFile][4], 2048, 144)

        if np.abs(startIndexTrue - loadArr[iDir][iFile][3]) <= accStartIndexErr:
            corrStartIndexCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1
        # if symIndexTrue == loadArr[iDir][iFile][5]:
        #     corrSymIndexCount[fileProtocolIndex][fileSNRIndex, fileCPOptIndex] += 1

# print(symIndexTrue == loadArr[iDir][iFile][5])
print(fileCount)
nData = 400
print(corrCPLenCount)
# print(corrStartIndexCount / nData)
# print(corrSymIndexCount / nData)

20
[array([[400., 400.],
        [400., 400.],
        [400., 400.],
        [400., 400.],
        [400., 400.],
        [400., 400.],
        [400., 400.],
        [400., 400.]]) array([[0., 0., 0.],
                              [0., 0., 0.],
                              [0., 0., 0.],
                              [0., 0., 0.],
                              [0., 0., 0.],
                              [0., 0., 0.],
                              [0., 0., 0.],
                              [0., 0., 0.]]) array([[200., 150.],
                                                    [200., 150.],
                                                    [200., 150.],
                                                    [200., 150.],
                                                    [200., 150.],
                                                    [200., 150.],
                                                    [200., 150.],
                                                    [200., 150.]])
 array